In [ ]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2


In [ ]:
import geopandas as gpd
import gfw
from pathlib import Path
import pandas as pd
from shapely import Polygon
from ast import literal_eval

In [ ]:
PATH = Path.cwd().parent.joinpath('data')

In [ ]:
# Get EEZ region

results = gfw.get_eez_list()

nl = [x for x in results if x.get('label') == 'Netherlands']
region = nl[0].get('id')


In [ ]:
# Create Polygons for ports

ijmuiden = ((4.504274, 52.493004),
          (4.910082, 52.449922),
          (4.89085588546282, 52.37956124860245),
          (4.394411, 52.413918),
          (4.504274, 52.493004))

rotterdam = ((3.892931,52.034854),
             (4.443900,51.956091),
             (4.423893,51.803771),
             (3.745890,51.899889),
             (3.892931,52.034854))

zeeland = ((3.299345,51.546999),
           (4.350941,51.467881),
           (4.273901,51.198332),
           (3.064373,51.403045),
           (3.299345,51.546999))

den_helder = ((4.641531,53.027905),
              (4.880262,53.010702),
              (4.892696,52.923835),
              (52.914838, 4.641531),
              (4.641531,53.027905))

urk = ((5.506152,52.730952),
       (5.690174,52.709111),
       (5.710574,52.598706),
       (5.535640,52.626007),
       (5.506152,52.730952)
       )

eemshaven = ((6.585831,53.652322),
             (7.044206,53.753703),
             (7.059318,53.266950),
             (6.578275,53.367753),
             (6.585831,53.652322))

harlingen = ((5.336631,53.214196),
             (5.525522,53.227767),
             (5.487744,53.129655),
             (5.319001,53.151561),
             (5.336631,53.214196))

eez = ((4.389826,53.336753),
       (2.980557,51.565663),
       (3.571947,51.381483),
       (4.402408,52.167808),
       (4.874262,52.864583),
       (5.704723,53.310448),
       (6.969290,53.297290),
       (6.560350,53.721935),
       (4.175919,53.548476),
       (4.389826,53.336753))

antwerpen = ((4.102983,51.395771),
             (4.349450,51.392949),
             (4.479467,51.224742),
             (4.195691,51.230406),
             (4.102983,51.395771))

zeebrugge = ((3.117470,51.369696),
             (3.260663,51.373926),
             (3.289121,51.296882),
             (3.133731,51.288972),
             (3.117470,51.369696))

oostende = ((2.879291,51.249601),
            (2.964360,51.258848),
            (2.979583,51.224091),
            (2.885559,51.206983),
            (2.879291,51.249601))

In [ ]:

geom = Polygon(antwerpen).__geo_interface__
path = PATH.joinpath('antwerpen.json')

In [ ]:
result = gfw.get_events_by_flag_and_geometry(flag='RUS', 
                                             start_date='2020-01-01',
                                             end_date='2024-01-01',
                                             geometry=geom,
                                             region=None,
                                             path_out=path,
                                             event_type='port_visits')

len(result.get('entries'))

## Parse data

In [ ]:
records = []

names = ['nl_pipes_loitering_4km']

for name in names:

    with open(PATH.joinpath('events', f'{name}.json'), 'r') as file:
        file = file.read()
        for entries in file:
            entries = literal_eval(entries)
            entries = entries.get('entries')
            for entry in entries:
                p = entry.get('position')
                v = entry.get('vessel')
                pv = entry.get('port_visit')

                record = {'start': entry.get('start'),
                        'end': entry.get('end'),
                        'id': entry.get('id'),
                        'lat': p.get('lat'),
                        'lon': p.get('lon'),
                        'vessel_id' : v.get('id'),
                        'vessel_name': v.get('name'),
                        'mmsi': v.get('ssvid'),
                        'type': v.get('type'),
                        'visit_id': pv.get('visit_id'),
                        'confidence': pv.get('confidence'),
                        'duration_hours': pv.get('durationHrs'),
                        'file': f'{name}.json'}
            
                for item in pv.keys():
                    if 'Anchorage' in item:
                        items = pv.get(item)
                        for key, value in items.items():
                            record.update({f'{item[0]}_{key}'.lower(): value})
                
                records.append(record)

df = pd.DataFrame(records)
df.drop_duplicates(subset=['vessel_id', 'start', 'end'], inplace=True)
len(df)

In [ ]:
df.to_csv(PATH.joinpath('port_visits_belgium.csv'), index=False)

## Get activity near infrastructure

Let's create polygons (buffers) around pipelines, cables and other infrastructure and see if we can find Russian vessels loitering there.

In [ ]:
pipes = PATH.joinpath('gis', 'mps_mapping_pipeline.gpkg')

In [ ]:
import fiona
fiona.listlayers(pipes)

In [ ]:
pipelines = gpd.read_file(PATH.joinpath('gis', 'mps_mapping_pipeline.gpkg'), layer='mps_mapping_pipeline')

In [ ]:
pipelines = pipelines[(pipelines.admin_area_name.isin(['Belgium', 'Netherlands', 'Denmark', 'Norway', 'United Kingdom', 'Germany'])) & (pipelines.mps_est_shore_status=='OFFSHORE')].copy()
len(pipelines)

In [ ]:
pipelines.to_csv(PATH.joinpath('gis', 'pipes_ns.csv'), index=False)

In [ ]:
pipelines.to_file(PATH.joinpath('gis', 'pipelines_for_gfw.geojson'), driver='GeoJSON')

In [ ]:
nl = pipelines[pipelines.admin_area_name=='Netherlands'].copy()
nl = nl.to_crs(28992)
nl = nl.geometry.buffer(2000)
nl = nl.reset_index(drop=True)

In [ ]:
nl = nl.to_crs(4326)

In [ ]:
for pipe in nl:
    gfw.get_events_by_flag_and_geometry(flag='RUS',
                                        start_date='2015-01-01',
                                        end_date='2023-12-31',
                                        event_type='loitering',
                                        geometry=pipe.__geo_interface__,
                                        path_out=PATH.joinpath('events', 'nl_pipes_loitering_4km.json'))
    